##### Usecase:Get sales data by orderstatus
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for each orderstatus
###### Problem statement-2:Get sales data for each orderstatus perday
###### Problem statement-3:Get total orders for each order status permonth
###### Problem statement-4:Get total orders for each order status peryear
###### Problem statement-5:Get monthly sales data for each order status for first quarter of 2014
###### Problem statement-6:Get monthly sales data for last quarter of 2013 with neither 'COMPLETE' nor 'CLOSED' orders
###### Problem statement-7:Get monthly sales data for last quarter of 2013 and first quarter of 2014  with 'CANCELED' orders
###### Problem statement-8:Get monthly sales data for 'PENDING' orders
###### Problem statement-9:Get monthly sales data for 'PROCESSING' OR ON_HOLD orders
###### Problem statement-10:Get monthly sales data with  'COMPLETE' AND 'CLOSED' orders

In [2]:
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql.functions import round,sum,avg,count,substring
from pyspark.sql.types import *
sqlContext = SQLContext(sc)
spark = SparkSession.builder.master('local').appName('getsalesdatabyorder').enableHiveSupport().getOrCreate()
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[6]: DataFrame[key: string, value: string]

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.registerTempTable("ordersTbl")
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.registerTempTable("orderItemsTbl")
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
sqlContext.sql("select * from ordersTbl").show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [6]:
sqlContext.sql("select * from orderItemsTbl").show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

##### Problem statement-1:Get sales data for each orderstatus

In [8]:
SalesDataPerorderPerstatus = sqlContext.sql('''select otl.order_status,round(sum(oitl.order_item_subtotal),2) as Totalrevenue_perstatus, 
                                                count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus 
                                                from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                                group by otl.order_status 
                                                order by otl.order_status''')
SalesDataPerorderPerstatus.coalesce(1).write.csv("/FileStore/tables/SalesDataPerorderPerstatus ", compression="none", header ='true')
SalesDataPerorderPerstatus.show(20,False)

+---------------+----------------------+-----------------------+-----------------------+
order_status |Totalrevenue_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+---------------+----------------------+-----------------------+-----------------------+
CANCELED |696031.0 |3519 |7702 |
CLOSED |3736048.86 |18668 |40510 |
COMPLETE |1.127693391E7 |56740 |123873 |
ON_HOLD |1864731.28 |9373 |20414 |
PAYMENT_REVIEW |357841.46 |1797 |4013 |
PENDING |3851881.36 |19291 |42260 |
PENDING_PAYMENT|7581671.2 |38031 |82935 |
PROCESSING |4190636.84 |20901 |45622 |
SUSPECTED_FRAUD|766844.69 |3878 |8429 |
+---------------+----------------------+-----------------------+-----------------------+

##### Problem statement-2:Get sales data for each orderstatus perday

In [10]:
SalesDataPerstatusPerday = sqlContext.sql('''select substring(otl.order_date,0,10) as orderdate, otl.order_status, 
                                             round(sum(oitl.order_item_subtotal),2) as Totalrevenue_perstatus,
                                             count(1) as Numberoforder_perstatus, oitl.order_item_quantity,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                             from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                             group by orderdate,otl.order_status,oitl.order_item_quantity 
                                             order by orderdate,otl.order_status''')
SalesDataPerstatusPerday.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPerday", compression="none", header ='true')
SalesDataPerstatusPerday.show(20,False)

+----------+--------------+----------------------+-----------------------+-------------------+-----------------------+
orderdate |order_status |Totalrevenue_perstatus|Numberoforder_perstatus|order_item_quantity|Totalquantity_perstatus|
+----------+--------------+----------------------+-----------------------+-------------------+-----------------------+
2013-07-25|CANCELED |429.97 |2 |1 |2 |
2013-07-25|CLOSED |7229.6 |32 |1 |32 |
2013-07-25|CLOSED |829.88 |7 |2 |14 |
2013-07-25|CLOSED |1007.8 |5 |4 |20 |
2013-07-25|CLOSED |599.88 |4 |3 |12 |
2013-07-25|CLOSED |1849.75 |6 |5 |30 |
2013-07-25|COMPLETE |1220.76 |9 |3 |27 |
2013-07-25|COMPLETE |9997.36 |48 |1 |48 |
2013-07-25|COMPLETE |1263.72 |12 |2 |24 |
2013-07-25|COMPLETE |2639.28 |12 |4 |48 |
2013-07-25|COMPLETE |4909.2 |15 |5 |75 |
2013-07-25|ON_HOLD |299.92 |3 |2 |6 |
2013-07-25|ON_HOLD |250.0 |1 |5 |5 |
2013-07-25|ON_HOLD |200.0 |1 |4 |4 |
2013-07-25|ON_HOLD |1149.92 |7 |1 |7 |
2013-07-25|PAYMENT_REVIEW|109.97 |2 |1 |2 |
2013-07-25|PAYMENT_REVIEW|749.85 |2 |5 |10 |
2013-07-25|PAYMENT_REVIEW|559.92 |2 |4 |8 |
2013-07-25|PENDING |367.92 |2 |4 |8 |
2013-07-25|PENDING |74.97 |1 |3 |3 |
+----------+--------------+----------------------+-----------------------+-------------------+-----------------------+
only showing top 20 rows

##### Problem statement-3:Get total orders for each order status permonth

In [12]:
SalesDataPerstatusPermonth = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               order by ordermonth,otl.order_status''')
SalesDataPerstatusPermonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPermonth", compression="none", header ='true')
SalesDataPerstatusPermonth.show(20,False)

+----------+---------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+---------------+--------------------+-----------------------+-----------------------+
2013-07 |CANCELED |9882.41 |58 |136 |
2013-07 |CLOSED |78107.44 |382 |833 |
2013-07 |COMPLETE |255358.01 |1291 |2880 |
2013-07 |ON_HOLD |37594.59 |199 |435 |
2013-07 |PAYMENT_REVIEW |7747.87 |43 |103 |
2013-07 |PENDING |86945.94 |419 |953 |
2013-07 |PENDING_PAYMENT|175973.91 |862 |1824 |
2013-07 |PROCESSING |97341.74 |493 |1097 |
2013-07 |SUSPECTED_FRAUD|15830.28 |74 |160 |
2013-08 |CANCELED |59422.67 |318 |730 |
2013-08 |CLOSED |314631.13 |1593 |3472 |
2013-08 |COMPLETE |907197.8 |4585 |10083 |
2013-08 |ON_HOLD |155155.39 |768 |1658 |
2013-08 |PAYMENT_REVIEW |25696.76 |139 |295 |
2013-08 |PENDING |345071.18 |1736 |3809 |
2013-08 |PENDING_PAYMENT|604406.7 |2998 |6627 |
2013-08 |PROCESSING |349112.79 |1786 |3889 |
2013-08 |SUSPECTED_FRAUD|67964.34 |345 |748 |
2013-09 |CANCELED |67973.94 |324 |687 |
2013-09 |CLOSED |338312.5 |1706 |3647 |
+----------+---------------+--------------------+-----------------------+-----------------------+
only showing top 20 rows

##### Problem statement-4:Get total orders for each order status peryear

In [14]:
SalesDataPerstatusPeryear = sqlContext.sql('''select substring(otl.order_date,0,4) as orderyear, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus, 
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               group by orderyear,otl.order_status
                                               order by orderyear,otl.order_status''')
SalesDataPerstatusPeryear.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPeryear", compression="none", header ='true')
SalesDataPerstatusPeryear.show(20,False)

+---------+---------------+--------------------+-----------------------+-----------------------+
orderyear|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+---------+---------------+--------------------+-----------------------+-----------------------+
2013 |CANCELED |314083.12 |1603 |3513 |
2013 |CLOSED |1713090.03 |8574 |18588 |
2013 |COMPLETE |4973802.1 |24951 |54616 |
2013 |ON_HOLD |829421.22 |4126 |9075 |
2013 |PAYMENT_REVIEW |143455.81 |742 |1594 |
2013 |PENDING |1715070.62 |8596 |18972 |
2013 |PENDING_PAYMENT|3386687.24 |17076 |36995 |
2013 |PROCESSING |1839024.53 |9224 |20281 |
2013 |SUSPECTED_FRAUD|339554.71 |1741 |3775 |
2014 |CANCELED |381947.89 |1916 |4189 |
2014 |CLOSED |2022958.83 |10094 |21922 |
2014 |COMPLETE |6303131.81 |31789 |69257 |
2014 |ON_HOLD |1035310.06 |5247 |11339 |
2014 |PAYMENT_REVIEW |214385.64 |1055 |2419 |
2014 |PENDING |2136810.73 |10695 |23288 |
2014 |PENDING_PAYMENT|4194983.96 |20955 |45940 |
2014 |PROCESSING |2351612.32 |11677 |25341 |
2014 |SUSPECTED_FRAUD|427289.99 |2137 |4654 |
+---------+---------------+--------------------+-----------------------+-----------------------+

##### Problem statement-5:Get monthly sales data for each order status for first quarter of 2014

In [16]:
SalesDataPerstatusfirstquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               where CAST(otl.order_date AS DATE)>='2014-01-01' AND CAST(otl.order_date AS DATE)<='2014-03-31'
                                               group by ordermonth,otl.order_status 
                                               order by ordermonth,otl.order_status''')
SalesDataPerstatusfirstquarter.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusfirstquarter", compression="none", header ='true')
SalesDataPerstatusfirstquarter.show(30,False)

+----------+---------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+---------------+--------------------+-----------------------+-----------------------+
2014-01 |CANCELED |53612.83 |263 |591 |
2014-01 |CLOSED |298236.04 |1492 |3247 |
2014-01 |COMPLETE |931985.73 |4706 |9958 |
2014-01 |ON_HOLD |181414.96 |894 |1913 |
2014-01 |PAYMENT_REVIEW |35816.65 |185 |404 |
2014-01 |PENDING |342057.67 |1699 |3818 |
2014-01 |PENDING_PAYMENT|670633.09 |3376 |7324 |
2014-01 |PROCESSING |346943.35 |1706 |3760 |
2014-01 |SUSPECTED_FRAUD|63746.75 |345 |754 |
2014-02 |CANCELED |65825.08 |318 |667 |
2014-02 |CLOSED |302636.06 |1547 |3251 |
2014-02 |COMPLETE |915134.06 |4667 |10251 |
2014-02 |ON_HOLD |148648.08 |754 |1642 |
2014-02 |PAYMENT_REVIEW |29113.01 |139 |329 |
2014-02 |PENDING |316896.06 |1576 |3486 |
2014-02 |PENDING_PAYMENT|612843.94 |3046 |6728 |
2014-02 |PROCESSING |327870.19 |1696 |3625 |
2014-02 |SUSPECTED_FRAUD|59697.24 |291 |659 |
2014-03 |CANCELED |57485.89 |292 |634 |
2014-03 |CLOSED |301673.84 |1514 |3344 |
2014-03 |COMPLETE |969677.16 |4880 |10642 |
2014-03 |ON_HOLD |162486.62 |805 |1687 |
2014-03 |PAYMENT_REVIEW |38871.57 |191 |431 |
2014-03 |PENDING |305423.94 |1513 |3343 |
2014-03 |PENDING_PAYMENT|601666.68 |3032 |6541 |
2014-03 |PROCESSING |353374.54 |1756 |3821 |
2014-03 |SUSPECTED_FRAUD|71832.03 |367 |778 |
+----------+---------------+--------------------+-----------------------+-----------------------+

##### Problem statement-6:Get monthly sales data for last quarter of 2013 with neither 'COMPLETE' nor 'CLOSED' orders

In [18]:
SalesDataPerstatuslastquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               where CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31'
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status!='COMPLETE' AND otl.order_status!='CLOSED'
                                               order by ordermonth,otl.order_status''')
SalesDataPerstatuslastquarter.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatuslastquarter", compression="none", header ='true')
SalesDataPerstatuslastquarter.show(30,False)

+----------+---------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+---------------+--------------------+-----------------------+-----------------------+
2013-10 |CANCELED |51024.94 |256 |560 |
2013-10 |ON_HOLD |144669.59 |735 |1616 |
2013-10 |PAYMENT_REVIEW |22879.35 |121 |278 |
2013-10 |PENDING |273922.9 |1397 |3102 |
2013-10 |PENDING_PAYMENT|566179.71 |2857 |6107 |
2013-10 |PROCESSING |340758.39 |1703 |3814 |
2013-10 |SUSPECTED_FRAUD|53478.84 |277 |581 |
2013-11 |CANCELED |62812.76 |318 |702 |
2013-11 |ON_HOLD |181362.63 |912 |1990 |
2013-11 |PAYMENT_REVIEW |27480.01 |147 |312 |
2013-11 |PENDING |364237.39 |1822 |4022 |
2013-11 |PENDING_PAYMENT|721418.19 |3664 |7890 |
2013-11 |PROCESSING |358435.04 |1773 |3840 |
2013-11 |SUSPECTED_FRAUD|72974.71 |375 |824 |
2013-12 |CANCELED |62966.39 |329 |698 |
2013-12 |ON_HOLD |147652.14 |716 |1591 |
2013-12 |PAYMENT_REVIEW |26757.83 |139 |298 |
2013-12 |PENDING |322284.07 |1635 |3527 |
2013-12 |PENDING_PAYMENT|682300.32 |3483 |7566 |
2013-12 |PROCESSING |353135.7 |1735 |3798 |
2013-12 |SUSPECTED_FRAUD|60148.25 |314 |661 |
+----------+---------------+--------------------+-----------------------+-----------------------+

###### Problem statement-7:Get monthly sales data for last quarter of 2013 and first quarter of 2014  with 'CANCELED' orders

In [20]:
SalesDatacanceledordersquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales,round(avg(oitl.order_item_subtotal),2) as Averagesales,
                                               count(1) as Numberoforders,sum(oitl.order_item_quantity) as Totalquantity
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               where CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2014-03-31'
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status =='CANCELED'
                                               order by ordermonth,otl.order_status''')
SalesDatacanceledordersquarter.coalesce(1).write.csv("/FileStore/tables/SalesDatacanceledordersquarter", compression="none", header ='true')
SalesDatacanceledordersquarter.show(30,False)

+----------+------------+----------+------------+--------------+-------------+
ordermonth|order_status|Totalsales|Averagesales|Numberoforders|Totalquantity|
+----------+------------+----------+------------+--------------+-------------+
2013-10 |CANCELED |51024.94 |199.32 |256 |560 |
2013-11 |CANCELED |62812.76 |197.52 |318 |702 |
2013-12 |CANCELED |62966.39 |191.39 |329 |698 |
2014-01 |CANCELED |53612.83 |203.85 |263 |591 |
2014-02 |CANCELED |65825.08 |207.0 |318 |667 |
2014-03 |CANCELED |57485.89 |196.87 |292 |634 |
+----------+------------+----------+------------+--------------+-------------+

###### Problem statement-8:Get monthly sales data for 'PENDING' orders

In [22]:
SalesDatapendingorder = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales,round(avg(oitl.order_item_subtotal),2) as Averagesales,
                                               count(1) as Numberoforders,sum(oitl.order_item_quantity) as Totalquantity
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status LIKE'%PEND%'
                                               order by ordermonth,otl.order_status''')
SalesDatapendingorder.coalesce(1).write.csv("/FileStore/tables/SalesDatapendingorder", compression="none", header ='true')
SalesDatapendingorder.show(30,False)

+----------+---------------+----------+------------+--------------+-------------+
ordermonth|order_status |Totalsales|Averagesales|Numberoforders|Totalquantity|
+----------+---------------+----------+------------+--------------+-------------+
2013-07 |PENDING |86945.94 |207.51 |419 |953 |
2013-07 |PENDING_PAYMENT|175973.91 |204.15 |862 |1824 |
2013-08 |PENDING |345071.18 |198.77 |1736 |3809 |
2013-08 |PENDING_PAYMENT|604406.7 |201.6 |2998 |6627 |
2013-09 |PENDING |322609.16 |203.28 |1587 |3559 |
2013-09 |PENDING_PAYMENT|636408.39 |198.13 |3212 |6981 |
2013-10 |PENDING |273922.9 |196.08 |1397 |3102 |
2013-10 |PENDING_PAYMENT|566179.71 |198.17 |2857 |6107 |
2013-11 |PENDING |364237.39 |199.91 |1822 |4022 |
2013-11 |PENDING_PAYMENT|721418.19 |196.89 |3664 |7890 |
2013-12 |PENDING |322284.07 |197.12 |1635 |3527 |
2013-12 |PENDING_PAYMENT|682300.32 |195.89 |3483 |7566 |
2014-01 |PENDING |342057.67 |201.33 |1699 |3818 |
2014-01 |PENDING_PAYMENT|670633.09 |198.65 |3376 |7324 |
2014-02 |PENDING |316896.06 |201.08 |1576 |3486 |
2014-02 |PENDING_PAYMENT|612843.94 |201.2 |3046 |6728 |
2014-03 |PENDING |305423.94 |201.87 |1513 |3343 |
2014-03 |PENDING_PAYMENT|601666.68 |198.44 |3032 |6541 |
2014-04 |PENDING |287068.62 |195.42 |1469 |3133 |
2014-04 |PENDING_PAYMENT|630317.76 |199.09 |3166 |6912 |
2014-05 |PENDING |296962.03 |198.64 |1495 |3286 |
2014-05 |PENDING_PAYMENT|612500.02 |202.28 |3028 |6720 |
2014-06 |PENDING |313181.44 |200.5 |1562 |3308 |
2014-06 |PENDING_PAYMENT|584000.5 |203.34 |2872 |6439 |
2014-07 |PENDING |275221.0 |199.29 |1381 |2914 |
2014-07 |PENDING_PAYMENT|483021.96 |198.37 |2435 |5276 |
+----------+---------------+----------+------------+--------------+-------------+

###### Problem statement-9:Get monthly sales data for 'PROCESSING' or ON_HOLD orders

In [24]:
SalesDataprocessingcanceledorders = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales,round(avg(oitl.order_item_subtotal),2) as Averagesales,
                                               count(1) as Numberoforders,sum(oitl.order_item_quantity) as Totalquantity
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status LIKE'%PROCESS%'OR otl.order_status LIKE'%HOLD%'
                                               order by ordermonth,otl.order_status''')
SalesDataprocessingcanceledorders.coalesce(1).write.csv("/FileStore/tables/SalesDataprocessingcanceledorders", compression="none", header ='true')
SalesDataprocessingcanceledorders.show(30,False)

+----------+------------+----------+------------+--------------+-------------+
ordermonth|order_status|Totalsales|Averagesales|Numberoforders|Totalquantity|
+----------+------------+----------+------------+--------------+-------------+
2013-07 |ON_HOLD |37594.59 |188.92 |199 |435 |
2013-07 |PROCESSING |97341.74 |197.45 |493 |1097 |
2013-08 |ON_HOLD |155155.39 |202.03 |768 |1658 |
2013-08 |PROCESSING |349112.79 |195.47 |1786 |3889 |
2013-09 |ON_HOLD |162986.86 |204.76 |796 |1785 |
2013-09 |PROCESSING |340240.88 |196.22 |1734 |3843 |
2013-10 |ON_HOLD |144669.59 |196.83 |735 |1616 |
2013-10 |PROCESSING |340758.39 |200.09 |1703 |3814 |
2013-11 |ON_HOLD |181362.63 |198.86 |912 |1990 |
2013-11 |PROCESSING |358435.04 |202.16 |1773 |3840 |
2013-12 |ON_HOLD |147652.14 |206.22 |716 |1591 |
2013-12 |PROCESSING |353135.7 |203.54 |1735 |3798 |
2014-01 |ON_HOLD |181414.96 |202.93 |894 |1913 |
2014-01 |PROCESSING |346943.35 |203.37 |1706 |3760 |
2014-02 |ON_HOLD |148648.08 |197.15 |754 |1642 |
2014-02 |PROCESSING |327870.19 |193.32 |1696 |3625 |
2014-03 |ON_HOLD |162486.62 |201.85 |805 |1687 |
2014-03 |PROCESSING |353374.54 |201.24 |1756 |3821 |
2014-04 |ON_HOLD |154928.83 |192.7 |804 |1767 |
2014-04 |PROCESSING |366710.81 |198.76 |1845 |3954 |
2014-05 |ON_HOLD |150186.95 |197.87 |759 |1618 |
2014-05 |PROCESSING |324532.19 |203.98 |1591 |3447 |
2014-06 |ON_HOLD |131820.94 |191.04 |690 |1511 |
2014-06 |PROCESSING |347167.08 |209.26 |1659 |3691 |
2014-07 |ON_HOLD |105823.66 |195.61 |541 |1201 |
2014-07 |PROCESSING |285014.18 |200.15 |1424 |3043 |
+----------+------------+----------+------------+--------------+-------------+

###### Problem statement-10:Get monthly sales data with  'COMPLETE' AND 'CLOSED' orders

In [26]:
SalesDatacompleteandclosedorders = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status=='COMPLETE' OR otl.order_status=='CLOSED'
                                               order by ordermonth,otl.order_status''')
SalesDatacompleteandclosedorders.coalesce(1).write.csv("/FileStore/tables/SalesDatacompleteandclosedorders", compression="none", header ='true')
SalesDatacompleteandclosedorders.show(30,False)

+----------+------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status|Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+------------+--------------------+-----------------------+-----------------------+
2013-07 |CLOSED |78107.44 |382 |833 |
2013-07 |COMPLETE |255358.01 |1291 |2880 |
2013-08 |CLOSED |314631.13 |1593 |3472 |
2013-08 |COMPLETE |907197.8 |4585 |10083 |
2013-09 |CLOSED |338312.5 |1706 |3647 |
2013-09 |COMPLETE |963943.33 |4826 |10536 |
2013-10 |CLOSED |294414.07 |1468 |3184 |
2013-10 |COMPLETE |877272.88 |4402 |9662 |
2013-11 |CLOSED |339567.76 |1693 |3725 |
2013-11 |COMPLETE |1040367.6 |5201 |11400 |
2013-12 |CLOSED |348057.15 |1732 |3727 |
2013-12 |COMPLETE |929662.48 |4646 |10055 |
2014-01 |CLOSED |298236.04 |1492 |3247 |
2014-01 |COMPLETE |931985.73 |4706 |9958 |
2014-02 |CLOSED |302636.06 |1547 |3251 |
2014-02 |COMPLETE |915134.06 |4667 |10251 |
2014-03 |CLOSED |301673.84 |1514 |3344 |
2014-03 |COMPLETE |969677.16 |4880 |10642 |
2014-04 |CLOSED |297959.39 |1463 |3133 |
2014-04 |COMPLETE |951764.16 |4838 |10600 |
2014-05 |CLOSED |294133.05 |1468 |3212 |
2014-05 |COMPLETE |927546.31 |4665 |10165 |
2014-06 |CLOSED |270021.59 |1341 |2966 |
2014-06 |COMPLETE |909732.49 |4465 |9938 |
2014-07 |CLOSED |258298.88 |1269 |2769 |
2014-07 |COMPLETE |697291.91 |3568 |7703 |
+----------+------------+--------------------+-----------------------+-----------------------+